In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Numeric, DateTime
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.automap import automap_base

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:

engine = create_engine(connection_string, echo=False)
meta = MetaData()

nome_tabela = 'COVID'

covid = Table('covid', meta, 
    Column("DATE", Date),
    Column("TYPE", String(30)),
    Column("CASES", Integer),
    Column("DIFFERENCE", Integer),
    Column("COUNTRY", String(50)),
    Column("PROVINCE_STATE", String(50)), 
    Column("LAT", Numeric(18,8)),
    Column("LONG", Numeric(18,8)),
    Column("LATEST_DATE", DateTime))

if not engine.dialect.has_table(engine, nome_tabela, schema = 'BLUADMIN'):
    meta.create_all(engine)
else:
    conn = engine.connect()
    #stmt = covid.delete().where(students.c.lastname == 'Khanna')
    stmt = covid.delete()
    conn.execute(stmt)
    conn.close()

In [4]:
Base = automap_base()

#engine = create_engine(connection_string, echo=True)
session = scoped_session(sessionmaker(autocommit=True,
                                      autoflush=False,
                                      bind=engine))

Base.prepare(engine, reflect=True)
base_sa = Base.classes

In [5]:
df = pd.read_csv('https://download.data.world/s/3xpcwdh7es3o6uqyztkkhdnh4ht7i7')
df.head()

,DATE,TYPE,CASES,DIFFERENCE,COUNTRY,PROVINCE_STATE,LAT,LONG,LATEST_DATE
0,2020-06-04,Confirmed,3262,152,Cote d'Ivoire,NaN,7.540000,-5.547100,2020-06-04T23:15:39
1,2020-06-04,Deaths,0,0,Eritrea,NaN,15.179400,39.782300,2020-06-04T23:15:39
2,2020-06-04,Deaths,1,0,France,French Guiana,4.000000,-53.000000,2020-06-04T23:15:39
3,2020-06-04,Confirmed,298,0,Canada,Manitoba,53.760900,-98.813900,2020-06-04T23:15:39
4,2020-06-04,Confirmed,20,0,France,New Caledonia,-20.904305,165.618042,2020-06-04T23:15:39


In [6]:
nome_tabela = 'covid'

In [7]:
df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False)
#df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False, chunksize=1000)   # With number of records to be processed each time

In [8]:
df2 = pd.read_sql_table(nome_tabela, engine)
df2.head()

,DATE,TYPE,cases,difference,country,province_state,lat,LONG,latest_date
0,2020-06-04,Deaths,10,0,Costa Rica,None,9.748900,-83.75340,2020-06-04 23:15:39
1,2020-06-04,Confirmed,25142,31,Ireland,None,53.142400,-7.69210,2020-06-04 23:15:39
2,2020-06-04,Confirmed,18,0,Belize,None,17.189900,-88.49760,2020-06-04 23:15:39
3,2020-06-04,Confirmed,2340,124,Kenya,None,-0.023600,37.90620,2020-06-04 23:15:39
4,2020-06-04,Deaths,12,0,Guyana,None,4.860416,-58.93018,2020-06-04 23:15:39


In [9]:
session.close()